In [1]:
import pandas as pd 
import regex as re
import unicodedata
import spacy
import nltk
from nltk.probability import FreqDist
import numpy as np
from nltk.stem import WordNetLemmatizer

from tqdm.auto import tqdm, trange
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To 

In [2]:
nlp_en = spacy.load('en_core_web_md')
nlp_es = spacy.load('es_core_news_md')

In [3]:
#Head Count
HC_columns = ['Status',
                'Role',
                'Agent Name' ,
                'Supervisor',	
                'ACCM',
                'Wave',
                'LOB'	,
                'GYG_ID',	
                'SITE',
                'Tenure Clasif ']

HC = pd.read_excel(r'HC.xlsx', sheet_name=1, usecols=HC_columns)

HC.dropna(subset=['GYG_ID'], inplace=True)
HC.drop_duplicates(subset=['GYG_ID'], inplace=True)
HC.to_csv('output/headcount.csv', index=False, sep=';', encoding='utf-8')
#HC.columns


In [4]:
HC['GYG_ID'].values

array(['gyg-norma.riossaldana@teleperformance.com',
       'gyg-katheryn.bohabotllerena@teleperformance.co',
       'gyg-adrian.molinacabala@teleperformance.co',
       'gyg-daniella.giraldoalvarez@teleperformance.co',
       'gyg-cesar.guzmanvera@teleperformance.co',
       'gyg-maria.soterocueto@teleperformance.co',
       'gyg-vania.mejiaolivera@teleperformance.co',
       'gyg-johana.luquepalacios@teleperformance.co',
       'gyg-grecia.pinedorengifo@teleperformance.co',
       'gyg-Carla.SequeirosRamos@teleperformance.co',
       'gyg-Leonardo.AguilarLazarte@teleperformance.co',
       'gyg-Jose.MarrouLuy@teleperformance.co',
       'gyg-fressia.sanchezdelacruz@teleperformance.co',
       'gyg-Pablo.AmasifuenSacsa@teleperformance.co',
       'gyg-Carmen.CanalesOre@teleperformance.co',
       'gyg-Sebastian.CornejoPonce@teleperformance.co',
       'gyg-Lizbeth.GuerreroGil@teleperformance.co',
       'gyg-Gabriel.DavilaBarraganes@teleperformance.co',
       'gyg-Maria.RojasRengifo@t

In [5]:
HC[HC['GYG_ID'].duplicated()]

,Status,Role,Agent Name,Supervisor,ACCM,Wave,LOB,GYG_ID,SITE,Tenure Clasif


In [6]:
columns=[   'Agents Email',
            'Freshdesk Ticket ID',
            'Date',
            'Freshdesk Ticket Source',
            'CSAT CSAT Rating Main Question',
            'CSAT Csat Feedback',
            'Freshdesk Ticket Language',
            'Contact Reason 1st  Requester',
            'Contact Reason 1st Intent',
            'Contact Reason 1st Theme',
            'Freshdesk Group Name',
            'CSAT Score']


import os 

filename=os.listdir('input')[0]
path = os.path.join('input', filename)

data = pd.read_excel(path, usecols=columns)


rename_columns= {'Agents Email' : 'Agent Email',
            'Freshdesk Ticket ID':'Survey ID',
            'Freshdesk Ticket Source':'Channel',
            'CSAT CSAT Rating Main Question': 'CSAT Score',
            'CSAT Csat Feedback':'Feedback',
            'Freshdesk Ticket Language':'Language',
            'Contact Reason 1st  Requester':'Requester',
            'Contact Reason 1st Intent':'Intent',
            'Contact Reason 1st Theme':'Theme',
            'Freshdesk Group Name':'Freshdesk Group',
            'CSAT Score':'% CSAT'}


            
data.rename(columns=rename_columns, inplace=True)

data['Feedback'] = data['Feedback'].replace('""','')
data.mask(data=='',inplace=True)

In [7]:
def get_NPS(x):
    if (x > 0):
        return "Promoter"
    elif (x == 0):
        return "Passive"
    else:
        return "Detractor"

In [8]:
before_preprocessing = len(data)
print('Data len before preprocessing', before_preprocessing)

data.dropna(subset=['CSAT Score'], inplace=True)
print('After removing NaN CSAT Score', len(data))

data['CSAT'] = data['CSAT Score'].apply(lambda x: get_NPS(x))
data.drop(['CSAT Score'], axis = 1, inplace = True)

Data len before preprocessing 39404
After removing NaN CSAT Score 39404


In [9]:
data.isnull().sum()

Agent Email            0
Survey ID              0
Date                   0
Channel                0
Feedback           33892
Language               0
Requester            968
Intent              1097
Theme               1097
Freshdesk Group        0
% CSAT                 0
CSAT                   0
dtype: int64

In [10]:
data.dropna(subset=['Survey ID'], inplace=True)
data['Survey ID']=data['Survey ID'].astype(str)+data['Agent Email'].astype(str)+data['Date'].astype(str)

In [11]:
data.columns

Index(['Agent Email', 'Survey ID', 'Date', 'Channel', 'Feedback', 'Language',
       'Requester', 'Intent', 'Theme', 'Freshdesk Group', '% CSAT', 'CSAT'],
      dtype='object')

In [12]:
print('Data len before preprocessing', before_preprocessing)



data.drop_duplicates(subset='Survey ID', inplace=True)
print('After removing SurveyID duplicates', len(data))

data = data[data['Agent Email']!='Merged']
print('After removing \'Merged\' data', len(data))

Data len before preprocessing 39404
After removing SurveyID duplicates 38600
After removing 'Merged' data 38530


In [13]:
data.isna().sum()

Agent Email            0
Survey ID              0
Date                   0
Channel                0
Feedback           33148
Language               0
Requester            624
Intent               687
Theme                687
Freshdesk Group        0
% CSAT                 0
CSAT                   0
dtype: int64

In [14]:
data.dtypes

Agent Email                object
Survey ID                  object
Date               datetime64[ns]
Channel                    object
Feedback                   object
Language                   object
Requester                  object
Intent                     object
Theme                      object
Freshdesk Group            object
% CSAT                      int64
CSAT                       object
dtype: object

In [15]:
data.dtypes

Agent Email                object
Survey ID                  object
Date               datetime64[ns]
Channel                    object
Feedback                   object
Language                   object
Requester                  object
Intent                     object
Theme                      object
Freshdesk Group            object
% CSAT                      int64
CSAT                       object
dtype: object

In [16]:
#surveys_raw = pd.read_csv('output/surveys_raw.csv', sep=';')

#surveys_raw = pd.read_excel('output/surveys_raw_backup.xlsx')

#surveys_raw.to_excel('output/surveys_raw_backup.xlsx', index=False)

#data=pd.concat([surveys_raw,data])

data[ "Date" ]=pd.to_datetime(data[ 'Date' ])

# Data since 01/01/2023
data.drop_duplicates(subset='Survey ID', keep='last', inplace=True)
data.to_csv('output/surveys_raw.csv', index=False, sep=';')

In [17]:
#Constuct the Data Frame with only SURVEY ID key 

df = data[['Survey ID', 'Feedback','Language']]

#Use only data with Comments
df = df[df['Feedback'].notnull()]
df['Feedback'] = df['Feedback'].apply(str)
df['Feedback'] = df['Feedback'].replace('""','')
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace('"',''))
df.head()
df_sentiment = df.copy()

#df_sentiment.to_excel('sentiment_raw.xlsx', index=False)


In [18]:
df['Feedback']=df['Feedback'].apply(lambda x: str(x).lower())

#Remove special characters

special_chars1 = df['Feedback'].apply(lambda x: [each for each in list(x) if not each.isalnum() and each != ' '] if type(x) == type('Hello') else '')
flat_list1 = [item for sublist in special_chars1 for item in sublist]
print(set(flat_list1)) # Set = List of character with no repeat
special_char_to_remove = set(flat_list1)

def remove_special_char(words):
    return [word for word in words if not str(word) in special_char_to_remove]

df

{'“', '(', '$', ',', ']', '_', '*', '’', ')', '\\', '❤', '\u200d', ';', '€', '#', "'", '‘', '&', '[', '¡', '\xa0', '«', '|', '⭐', '/', '+', '£', '„', '✨', '>', '☺', '!', '…', '！', '♡', '@', '–', '-', ':', '♥', '¿', '️', '•', '✌', '♀', '。', '☹', '”', '、', '~', '<', '=', '%', '`', '.', '»', '\u3000', '—', '´', '?'}


,Survey ID,Feedback,Language
6,12963500gyg-valeria.vasquezmiranda@teleperform...,i got help from valeria vasquez (amoung others...,English
13,12990999gyg-fressia.sanchezdelacruz@teleperfor...,everything was great!,English
18,13002836gyg-karla.chungvelasquez@teleperforman...,don’t understand how you can charge someone tw...,English
19,13016161gyg-cinthya.santosmoran@teleperformanc...,issue an invoice with vat id,English
25,13014113gyg-cinthya.santosmoran@teleperformanc...,"asking how to download barcode for swiss pass,...",English
...,...,...,...
39381,17135621gyg-renato.valdiviasolier@teleperforma...,la primera vez que me comunique con ustedes fu...,Spanish
39383,17616079gyg-alessandra.altamiranopacheco@telep...,nothing the person was excellent in all criteria,English
39393,17627457gyg-alondra.chamochumbivicuna@teleperf...,seriously the best customer service i have exp...,English
39396,17592259angelo.guerrerolinares@teleperformance...,very helpful and friendly indeed! thanks!,English


In [19]:
##  No se le pueden quitar las tildes porque si no el npl_es no lematiza 


df['Feedback']=df['Feedback'].apply(lambda x: str(x).lower())

#Remove special characters

special_chars1 = df['Feedback'].apply(lambda x: [each for each in list(x) if not each.isalnum() and each != ' '] if type(x) == type('Hello') else '')
flat_list1 = [item for sublist in special_chars1 for item in sublist]
print(set(flat_list1)) # Set = List of character with no repeat
special_char_to_remove = set(flat_list1)

def remove_special_char(words):
    return [word for word in words if not str(word) in special_char_to_remove]

df['spacy']=df['Feedback'].apply(lambda x: remove_special_char(nlp_es(str(x)))).where(df['Language'] == 'Spanish')
df['spacy']=df['Feedback'].apply(lambda x: remove_special_char(nlp_en(str(x)))).where(df['Language'] == 'English', other = df['spacy'])

df.dropna(inplace=True, subset=['spacy'])

df['Lemmatization'] = df['spacy'].apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space and len(token) > 2 and token.is_alpha]) # doc if str(doc) == 'nan' else # not token.is_stop and



df_cat = df.copy()

df_cat['joint_lemma_words'] = df_cat['Lemmatization'].apply(lambda x: ' '.join(x))
df_cat.drop(['Lemmatization', 'Feedback', 'Language','spacy'], axis = 1, inplace = True)


df_cat.to_excel('categorization_test.xlsx', index=False)

df.to_csv("output/freqwords.csv",index=False, sep=';', encoding='utf-8')


{'“', '(', '$', ',', ']', '_', '*', '’', ')', '\\', '❤', '\u200d', ';', '€', '#', "'", '‘', '&', '[', '¡', '\xa0', '«', '|', '⭐', '/', '+', '£', '„', '✨', '>', '☺', '!', '…', '！', '♡', '@', '–', '-', ':', '♥', '¿', '️', '•', '✌', '♀', '。', '☹', '”', '、', '~', '<', '=', '%', '`', '.', '»', '\u3000', '—', '´', '?'}


In [20]:
# Drop 'Feedback' and 'spacy' columns
df.drop(['Feedback', 'spacy', 'Language'], axis=1, inplace=True)

#Super Important! 
df.reset_index(drop=True, inplace=True)

df_bigram=df.copy()
df_trigram=df.copy()


# Common words

In [21]:
df["frecwords"]=df['Lemmatization'].apply(lambda x: FreqDist(x))
df=df.drop(['Lemmatization'], axis = 1)

df_temp = pd.DataFrame(pd.DataFrame(df["frecwords"].values.tolist()).stack().reset_index(level=1))
df_temp.columns = ['Words', 'Frequency']
df= df.join(df_temp)

df.drop(["frecwords"], axis = 1, inplace = True)

# Removing unnecesary blankspaces around words
df['Words'] = df['Words'].apply(lambda x: str(x).strip())

df.dropna(inplace=True)
df['Frequency'] = df['Frequency'].astype('int8')
df.to_csv(r"output/Words.csv", encoding = 'utf-8', index = False, sep=';')

df.head()

,Survey ID,Words,Frequency
0,12963500gyg-valeria.vasquezmiranda@teleperform...,get,1
0,12963500gyg-valeria.vasquezmiranda@teleperform...,help,1
0,12963500gyg-valeria.vasquezmiranda@teleperform...,valeria,1
0,12963500gyg-valeria.vasquezmiranda@teleperform...,vasquez,1
0,12963500gyg-valeria.vasquezmiranda@teleperform...,amoung,1


# Bigrams

In [22]:
def common_bigrams(words_list):
    list_bigrams = list(nltk.bigrams(words_list))
    filtered_bigram_dist = FreqDist(list_bigrams)
    return dict(filtered_bigram_dist)

In [23]:

df_bigram['bigrams_dict'] = df_bigram['Lemmatization'].apply(lambda x: common_bigrams(x))

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
df_temp_bigrams = pd.DataFrame(pd.DataFrame(df_bigram['bigrams_dict'].values.tolist()).stack().reset_index(level=1))
df_temp_bigrams.columns = ['Bigrams_tuple', 'bigrams_count']

display(df_temp_bigrams)
# Split bigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_temp_bigrams[['bigram_1', 'bigram_2']] = pd.DataFrame(df_temp_bigrams['Bigrams_tuple'].tolist(), index=df_temp_bigrams.index)

df_bigram = df_bigram.join(df_temp_bigrams)

df_bigram.drop(['Lemmatization', 'bigrams_dict','Bigrams_tuple' ], axis = 1, inplace = True)
df_bigram['Bigrams'] = df_bigram['bigram_1'] +" " +df_bigram['bigram_2']
df_bigram.dropna(subset=['bigrams_count'], inplace=True)
df_bigram['bigrams_count'] = df_bigram['bigrams_count'].astype('int8')
df_bigram.to_csv("output/Bigrams.csv",index=False, sep=';', encoding='utf-8')
df_bigram.head()

,Bigrams_tuple,bigrams_count
0,"(get, help)",1.0
0,"(help, valeria)",1.0
0,"(valeria, vasquez)",1.0
0,"(vasquez, amoung)",1.0
0,"(amoung, fast)",1.0
...,...,...
5300,"(seriously, good)",1.0
5301,"(helpful, friendly)",1.0
5301,"(friendly, thank)",1.0
5302,"(perfecto, solucionado)",1.0


,Survey ID,bigrams_count,bigram_1,bigram_2,Bigrams
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,get,help,get help
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,help,valeria,help valeria
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,valeria,vasquez,valeria vasquez
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,vasquez,amoung,vasquez amoung
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,amoung,fast,amoung fast


# Trigrams

In [24]:
def common_trigrams(words_list):
    list_trigrams = list(nltk.trigrams(words_list))
    filtered_trigrams_dist = FreqDist(list_trigrams)
    return dict(filtered_trigrams_dist)

In [25]:
df_trigram_backup=df_trigram

In [26]:
df_trigram = df_trigram_backup

In [27]:

df_trigram['trigrams_dict'] = df_trigram['Lemmatization'].apply(lambda x: common_trigrams(x))

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
df_temp_trigrams = pd.DataFrame(pd.DataFrame(df_trigram['trigrams_dict'].values.tolist()).stack().reset_index(level=1))
df_temp_trigrams.columns = ['Trigrams_tuple', 'Trigrams_count']

display(df_temp_trigrams)

df_temp_trigrams[['trigram_1', 'trigram_2', 'trigram_3']] = pd.DataFrame(df_temp_trigrams['Trigrams_tuple'].tolist(), index=df_temp_trigrams.index)

df_trigram = df_trigram.join(df_temp_trigrams)

df_trigram.drop(['Lemmatization', 'trigrams_dict','Trigrams_tuple' ], axis = 1, inplace = True)
df_trigram['Trigrams'] = df_trigram['trigram_1'] +" " +df_trigram['trigram_2'] +" " +df_trigram['trigram_3']
df_trigram.dropna(subset=['Trigrams_count'], inplace=True)
df_trigram['Trigrams_count'] = df_trigram['Trigrams_count'].astype('int8')
df_trigram.to_csv("output/Trigrams.csv",index=False, sep=';', encoding='utf-8')
df_trigram.head()

,Trigrams_tuple,Trigrams_count
0,"(get, help, valeria)",1.0
0,"(help, valeria, vasquez)",1.0
0,"(valeria, vasquez, amoung)",1.0
0,"(vasquez, amoung, fast)",1.0
0,"(amoung, fast, efficent)",1.0
...,...,...
5300,"(good, customer, service)",1.0
5300,"(customer, service, experience)",1.0
5300,"(seriously, good, customer)",1.0
5301,"(helpful, friendly, thank)",1.0


,Survey ID,Trigrams_count,trigram_1,trigram_2,trigram_3,Trigrams
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,get,help,valeria,get help valeria
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,help,valeria,vasquez,help valeria vasquez
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,valeria,vasquez,amoung,valeria vasquez amoung
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,vasquez,amoung,fast,vasquez amoung fast
0,12963500gyg-valeria.vasquezmiranda@teleperform...,1,amoung,fast,efficent,amoung fast efficent


# Sentiment

In [28]:
######################### Model settings ############################
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
######################################################################


########## Preprocess text (username and link placeholders) #########
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
#####################################################################


######################### Use model #################################
def get_sentiment_scores(tweet):
    #text = preprocess(tweet)
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores
######################################################################

########################################### Get sentiment #######################################################
tokenizer.model_max_length = 1028
lst_negative_scores = []
lst_neutral_scores = []
lst_positive_scores = []
lst_label = []

df_sentiment['Feedback'] = df_sentiment['Feedback'].astype('str')
df_sentiment=df_sentiment[df_sentiment['Feedback'].str.len() < 1500]

for i in trange(df_sentiment.shape[0]):
    tweet_i = str(df_sentiment['Feedback'].values[i])
    
    scores_array = get_sentiment_scores(tweet_i)
    negative_score, neutral_score, positive_score = scores_array
    lst_negative_scores.append(negative_score)
    lst_neutral_scores.append(neutral_score)
    lst_positive_scores.append(positive_score)
    
    ranking = np.argsort(scores_array)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    lst_label.append(label)
    
df_sentiment['NEGATIVE_SCORE'] = lst_negative_scores
df_sentiment['NEUTRAL_SCORE'] = lst_neutral_scores
df_sentiment['POSITIVE_SCORE'] = lst_positive_scores
df_sentiment['LABEL'] = lst_label



df_sentiment['NEGATIVE_SCORE'] = df_sentiment['NEGATIVE_SCORE'].astype('str')
df_sentiment['NEGATIVE_SCORE'] = df_sentiment['NEGATIVE_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment['NEUTRAL_SCORE'] = df_sentiment['NEUTRAL_SCORE'].astype('str')
df_sentiment['NEUTRAL_SCORE'] = df_sentiment['NEUTRAL_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment['POSITIVE_SCORE']=df_sentiment['POSITIVE_SCORE'].astype('str')
df_sentiment['POSITIVE_SCORE'] = df_sentiment['POSITIVE_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment.drop(['Feedback','Language'], axis = 1, inplace = True)

df_sentiment.to_csv('output/Sentiment.csv', index=False, sep=';')
df_sentiment


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/5375 [00:00<?, ?it/s]

,Survey ID,NEGATIVE_SCORE,NEUTRAL_SCORE,POSITIVE_SCORE,LABEL
6,12963500gyg-valeria.vasquezmiranda@teleperform...,"0,005291904","0,024448082","0,97026",positive
13,12990999gyg-fressia.sanchezdelacruz@teleperfor...,"0,0043757875","0,011321556","0,9843025",positive
18,13002836gyg-karla.chungvelasquez@teleperforman...,"0,88507617","0,109285355","0,005638282",negative
19,13016161gyg-cinthya.santosmoran@teleperformanc...,"0,0722204","0,86461943","0,06316031",neutral
25,13014113gyg-cinthya.santosmoran@teleperformanc...,"0,2217109","0,7468425","0,031446643",neutral
...,...,...,...,...,...
39381,17135621gyg-renato.valdiviasolier@teleperforma...,"0,040024474","0,839106","0,12086947",neutral
39383,17616079gyg-alessandra.altamiranopacheco@telep...,"0,057740375","0,19127913","0,75098044",positive
39393,17627457gyg-alondra.chamochumbivicuna@teleperf...,"0,0066578286","0,012651858","0,9806903",positive
39396,17592259angelo.guerrerolinares@teleperformance...,"0,006288374","0,014729161","0,9789824",positive


# Categorization

In [29]:
df_agent = df_cat.copy()
df_brand = df_cat.copy()
df_channel = df_cat.copy()

In [30]:
def remove_nonalpha(words):
    list_nonalpha = [w.strip() for w in words if w.strip().isalpha()]        
    return list_nonalpha 
    
def get_category(x):
    key_list = []
    match_list = []
    for key in categories_dict.keys():
        for value in categories_dict[key]:
            if (bool(re.search(value,x)) ==True):
                if key not in key_list:
                    key_list.append(key)
                    match_list.append(re.compile(value).findall(x))
    return dict(zip(key_list, match_list))

In [31]:

#Agent 

categories_dict = {
    "Show Expertise": [
        '(agent|rep| he| she|nobody|you|they|still|person|support|helpline)?(\s|\w+\s|\s\w+\s){0,3}(didn|wasn|not|no)(\s|\w+\s|\s\w+\s){0,3}(know|unexpert|expert)',
        '(agent|rep| he| she|nobody|you|they|still|person|support|helpline)?(\s|\w+\s|\s\w+\s){0,3}(new|hire|rookie)',
    ], 
    "Unwilling to Help": [
        '(speak|talk|ask|request)(\s|\w+\s|\s\w+\s){0,3}(supervisor|manager)',
        '(speak|talk|ask|request)(\s|\w+\s|\s\w+\s){0,3}(supervisor|manager)',
        '(didn|wasn|not)(\s|\w+\s|\s\w+\s){0,3}(pass|let|speak|talk)(\s|\w+\s|\s\w+\s){0,3}(supervisor|manager)',
        '(didn|wasn|not|no)(\s|\w+\s|\s\w+\s){0,3}(supervisor|manager)(\s|\w+\s|\s\w+\s){0,3}(available)'
    ],
    "Impolite": [
        '(agent|rep|she|he|be)(\s|\w+\s|\s\w+\s){0,5}(rude|disrespectful|impolite|disrespect|(over talk)|(talk over)|unprofessional|negligent|stubborn|argue|yell)',
        '(rude|disrespectful|impolite|disrespect|(over talk)|(talk over)|unprofessional|negligent|stubborn)(\s|\w+\s|\s\w+\s){0,5}(agent|rep|she|help|he)',
        '(no|zero)(\s|\w+\s|\s\w+\s){0,5}(empathy)'
    ],
    "Hang Up": [
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(hang)(\s|\w+\s|\s\w+\s){0,3}(up)', 
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(finish|end|disconnect|return|drop)(\s|\w+\s|\s\w+\s){0,3}(call|chat)', 
        '(be)(\s|\w+\s|\s\w+\s){0,3}(hang)(\s|\w+\s|\s\w+\s){0,3}(up)',
        '(be)(\s|\w+\s|\s\w+\s){0,3}(disconnec|dropped)'   
    ],
    "Knowledgeable": [
        '(agent|rep|he|she|nobody|you|they|still|person|support|helpline)(\s|\w+\s|\s\w+\s){0,3}( no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(fast|slow|quick|rapid|rapidly|quickly|speedly )',
        '(never|didn|not)(\s|\w+\s|\s\w+\s){0,3}(call|contact|touch|email)(\s|\w+\s|\s\w+\s){0,3}(back)',
        '(self|myselft|service|selfservice)(\s|\w+\s|\s\w+\s){0,3}(service|myself|option|chance|alternative|resource|possibility)',
        '(incorrect|miss|incomplete|wrong|inexact|inaccurate|imprecise|erroneous)(\s|\w+\s|\s\w+\s){0,3}(information|detail|fact|data|guidance|instruction|knowledge)',
        '(still|i|we)(\s|\w+\s|\s\w+\s){0,3}(need)(\s|\w+\s|\s\w+\s){0,3}(help|solution|answer|resolution|fix)',
        '(agent|rep|he|she|nobody|you |they|still|person|support|helpline)(\s|\w+\s|\s\w+\s){0,3}(any|zero|no)(\s|\w+\s|\s\w+\s){0,3}(help|resolution|solution|support|assistance)',
        '(help|solve|fix|address|answer|handle|resolve)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question)',
        '(no|didn|hasn|wasn|actually)(\s|\w+\s|\s\w+\s){0,3}(get|receive)(\s|\w+\s|\s\w+\s){0,3}(answer|help|resolution|solution|fix)',
        '(he|she|agent|rep|person)(\s|\w+\s|\s\w+\s){0,3}(didn|not|couldn|wasn|hasn|no|zero)(\s|\w+\s|\s\w+\s){0,3}(solve|answer|help|handle|fix|refund|cancel|clear)',
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(did|do)(\s|\w+\s|\s\w+\s){0,3}(nothing)',
        '(issue|problem|concern|case|complain|question)(\s|\w+\s|\s\w+\s){0,3}(no|wasn|weren|didn|not|hasn)(\s|\w+\s|\s\w+\s){0,3}(solve|fix|handle|address|answer)',
        '(didn|wasn|not)(\s|\w+\s|\s\w+\s){0,3}(solve|fix|handle|address|answer)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question)',
        '(issue|problem|concern|case|complain)(\s|\w+\s|\s\w+\s){0,3}(unresolve|unfix|unhandle)',
        '(actually)(\s|\w+\s|\s\w+\s){0,3}(help|solve|fix|address|answer|handle|resolve)',
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(lie)',
        '(follow)(\s|\w+\s|\s\w+\s){0,3}(up)',
        '(never|didn|not)(\s|\w+\s|\s\w+\s){0,3}(call|contact|touch|email)(\s|\w+\s|\s\w+\s){0,3}(back)',
        '(never)(\s|\w+\s|\s\w+\s){0,3}(receive|get|return)(\s|\w+\s|\s\w+\s){0,3}(call|email|chat)'
    ],
    "Issue Resolution": [
        '(agent|rep|he|she|nobody|you|they|still|person|support|helpline)(\s|\w+\s|\s\w+\s){0,3}( no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(resolve|handle|address|solve|fix|help|answer|clear )',
        '(issue|problem|concern|case|complain)(\s|\w+\s|\s\w+\s){0,3}(unresolve|unfix|unhandle)',
        '(still|i|we)(\s|\w+\s|\s\w+\s){0,3}(need)(\s|\w+\s|\s\w+\s){0,3}(help|solution|answer|resolution|fix)',
        '(agent|rep|he|she|nobody|you |they|still|person|support|helpline)(\s|\w+\s|\s\w+\s){0,3}(any|zero|no)(\s|\w+\s|\s\w+\s){0,3}(help|resolution|solution|support|assistance)',
        '(actually)(\s|\w+\s|\s\w+\s){0,3}(help|solve|fix|address|answer|handle|resolve)',
        '(help|solve|fix|address|answer|handle|resolve)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question)',
        '(no|didn|hasn|wasn|actually)(\s|\w+\s|\s\w+\s){0,3}(get|receive)(\s|\w+\s|\s\w+\s){0,3}(answer|help|resolution|solution|fix)',
        '(he|she|agent|rep|person)(\s|\w+\s|\s\w+\s){0,3}(didn|not|couldn|wasn|hasn|no|zero)(\s|\w+\s|\s\w+\s){0,3}(solve|answer|help|handle|fix|refund|cancel|clear)',
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(did|do)(\s|\w+\s|\s\w+\s){0,3}(nothing)',
        '(issue|problem|concern|case|complain|question)(\s|\w+\s|\s\w+\s){0,3}(no|wasn|weren|didn|not|hasn)(\s|\w+\s|\s\w+\s){0,3}(solve|fix|handle|address|answer)',
        '(didn|wasn|not)(\s|\w+\s|\s\w+\s){0,3}(solve|fix|handle|address|answer)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question)',
        '(didn|wasn|not)(\s|\w+\s|\s\w+\s){0,3}(cancel|refund)(\s|\w+\s|\s\w+\s){0,3}(order|item)',
        '(issue|problem|concern|case|complain)(\s|\w+\s|\s\w+\s){0,3}(unresolve|unfix|unhandle)',
        '(actually)(\s|\w+\s|\s\w+\s){0,3}(help|solve|fix|address|answer|handle|resolve)',
        '(speak|talk)(\s|\w+\s|\s\w+\s){0,3}(manager|supervisor)',
        '(agente|el|ella|rep|ejecutiva)(\s|\w+\s|\s\w+\s){0,3}(no)(\s|\w+\s|\s\w+\s){0,3}(resolver|ayudar|responder|solucion|solución|solucin|solucionar|arreglar|atender|asistir|colaborar|asistencia)',
        '(\s|\w+\s|\s\w+\s){1,3}(no|poder|querer|dar|nadie|brindar)(\s|\w+\s|\s\w+\s){0,3}(resolver|ayudar|respuesta|solucion|solución|solucin|arreglar|atender|asistir|asistencia)',
        '(\s|\w+\s|\s\w+\s){1,3}(solucion|solución|solucionar|solucin|ayuda|respuesta|resolver|arreglar)(\s|\w+\s|\s\w+\s){0,3}(nada|problema|inquietud|necesidad)',
        '(dar)(\s|\w+\s|\s\w+\s){0,3}(solucion|solución|solucionar|solucin|ayuda|respuesta|resolver|asistencia|asisitir|colaborar)',
        '(problema|pregunta|inconveniente)(\s|\w+\s|\s\w+\s){0,3}(no|ningun)(\s|\w+\s|\s\w+\s){0,3}(solucion|solución|solucionar|ayuda|respuesta|resolver|arreglar)',
        '(nada|no|ningun)(\s|\w+\s|\s\w+\s){0,3}(solucion|solución|solucionar|solucin|ayuda|respuesta|resolver|arreglar)'
    ],
    "Communication Skills": [
        '(speak|talk|understad|bad|poor|terrible|horrible)(\s|\w+\s|\s\w+\s){0,3}(english|language|clear)',
        '(hard|difficult|couldn)(\s|\w+\s|\s\w+\s){0,3}(understand)','(hard|heavy)(\s|\w+\s|\s\w+\s){0,3}(accent)',
        '(language)(\s|\w+\s|\s\w+\s){0,3}(barrier)','(listen|understand|read)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question|message)',
        '(no)(\s|\w+\s|\s\w+\s){0,3}(good)(\s|\w+\s|\s\w+\s){0,3}(listen)',
        '(english)(\s|\w+\s|\s\w+\s){0,3}(no |not |wasn)(\s|\w+\s|\s\w+\s){0,3}(language)',
        '(speak|understad|bad|poor|terrible|horrible|break|comprehend|write|read)(\s|\w+\s|\s\w+\s){0,3}(english|language)',
        '(hard|difficult|couldn|imposible)(\s|\w+\s|\s\w+\s){0,3}(understand)(\s|\w+\s|\s\w+\s){0,3}(him|her|rep|agent|english|accent|each)',
        '(hard|heavy|strong|foreign|break|latin|spanish)(\s|\w+\s|\s\w+\s){0,3}(accent)','(hard|difficult|couldn|impossible)(\s|\w+\s|\s\w+\s){0,3}(understand)(\s|\w+\s|\s\w+\s){0,3}(she|he)(\s|\w+\s|\s\w+\s){0,3}(say)',
        '(language)(\s|\w+\s|\s\w+\s){0,3}(barrier)','(english)(\s|\w+\s|\s\w+\s){0,3}(first|second)(\s|\w+\s|\s\w+\s){0,3}(language)', 
        '(english)(\s|\w+\s|\s\w+\s){0,3}(poor|bad|break|horrible|terrible)',
         "(representante, hablar)","(persona, hablar)","(difícil, entender)",
        "(podría, entender)","(entender, pregunta)","(difícil, tiempo)","(conseguir, alguien)",
        "(hablar, inglés)","(preguntar, hablar)","(entender, problema)","(pesado, acento)",
        "(entender, tema)","(fácil, entender)","(difícil, entender)","(entender, acento)",
        "(contratar, gente)","(entender, inglés)","(rep, hablar)","(dificultad, entender)",
        "(entender, representante)","(entender, vrbo)","(tiempo, entender)","(entender, decir)",
        "(señor, habla)","(dificultad, entender)","(señora,hablar)"         
    ],
    "Soft Skills": [
        '(pesimo|pésimo|mal|malo|horrible|nefasto|ineficiente|insuficiente)(\s|\w+\s|\s\w+\s){0,3}(servicio|atencion|soporte|atención)(\s|\w+\s|\s\w+\s){1,3}',
        '(agente|el|ella|rep|ejecutiva)(\s|\w+\s|\s\w+\s){0,5}(groser|rudoinepto|irrespetuoso|(mal(\s|\w+\s|\s\w+\s){0,3}educado))',
        '(poco|no|cero)(\s|\w+\s|\s\w+\s){0,3}(profesional)',
        '(\s|\w+\s|\s\w+\s){1,3}(irrespetuoso)',
        '(\s|\w+\s|\s\w+\s){1,5}(empatia|empatico|empatía|empatica)'
    ],
    "Contact Issues": [
        '(agent|rep| he| she|nobody|you|they|still|person|support|helpline)?(\s|\w+\s|\s\w+\s){0,3}(end |close|hang|hung|disconnect|leave)(\s|\w+\s|\s\w+\s){0,3}(chat|call|interaction|message)',
        '(long|excesive|much|put)(\s|\w+\s|\s\w+\s){0,3}(hold)',
        '(agent|rep| he| she|nobody| you|they|still|person|support|helpline)(\s|\w+\s|\s\w+\s){0,3}(hung|hang)(\s|\w+\s|\s\w+\s){0,3}(up)',
        '(wait|queue)(\s|\w+\s|\s\w+\s){0,3}(speak|hold)(\s|\w+\s|\s\w+\s){0,4}(minutes|mins |min )',
        '(speak|talk|ask|request)(\s|\w+\s|\s\w+\s){0,3}(supervisor|manager)',
        '(\s|\w+\s|\s\w+\s){1,3}(colgar|finalizar|cerrar|acabar|desconectar|sacar)(\s|\w+\s|\s\w+\s){0,3}(llamada|servicio|mensaje|chat|email|comunicacion|comunicación|interaccion|interacción|ticket|caso|conversación|conversacion|atención|atencion)', 
        '(\s|\w+\s|\s\w+\s){1,3}(desconectar|cortar|cerrar|colgar)(\s|\w+\s|\s\w+\s){1,3}',
        '(\s|\w+\s|\s\w+\s){1,3}(espera|hold|silencio)(\s|\w+\s|\s\w+\s){0,3}(minutos|rato|tiempo|hora)',
        '(hablar|comunicar|platicar|conversar|solicitar|querer|pedir|necesitar|negar)(\s|\w+\s|\s\w+\s){0,3}(manager|supervisor|jefe|superior)'
    ],
    
    "Understand Customer": [
        '(listen|understand|read)(\s|\w+\s|\s\w+\s){0,3}(issue|problem|concern|case|complain|question|message|information)',
        '(not|wasn)(\s|\w+\s|\s\w+\s){0,3}(good)(\s|\w+\s|\s\w+\s){0,3}(listen)',
        '(repeat)(\s|\w+\s|\s\w+\s){0,3}(myself)'
    ],
    
    "RCR": [
        '(never)(\s|\w+\s|\s\w+\s){0,3}(receive|get|return)(\s|\w+\s|\s\w+\s){0,3}(call|email|chat)',
        '(never|didn|not)(\s|\w+\s|\s\w+\s){0,3}(call|contact|touch|email)(\s|\w+\s|\s\w+\s){0,3}(back)',
        '(be|he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(tell)(\s|\w+\s|\s\w+\s){0,3}(get|receive)(\s|\w+\s|\s\w+\s){0,3}(call|email)',
        '(be|he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(tell)(\s|\w+\s|\s\w+\s){0,3}(order)(\s|\w+\s|\s\w+\s){0,3}(would|cancel|refund|deliver|ready|receive|send|be|pick)',
        '(be|he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(tell)(\s|\w+\s|\s\w+\s){0,3}(cancel|refund|get|receive)',
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(hang)(\s|\w+\s|\s\w+\s){0,3}(up)', 
        '( he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(finish|end|disconnect|return)(\s|\w+\s|\s\w+\s){0,3}(call|chat)', 
        '(be)(\s|\w+\s|\s\w+\s){0,3}(hang)(\s|\w+\s|\s\w+\s){0,3}(up)',
        '(be)(\s|\w+\s|\s\w+\s){0,3}(disconnec)','(hold)(\s|\w+\s|\s\w+\s){0,3}(minutes)',
        '(follow)(\s|\w+\s|\s\w+\s){0,3}(up)',
        '(he|she|agent|rep)(\s|\w+\s|\s\w+\s){0,3}(lie)'
    ],

    "Unprofessional": [
        '(agent|rep|she|he|be)(\s|\w+\s|\s\w+\s){0,5}(rude|disrespectful|impolite|disrespect|(over talk)|(talk over)|unprofessional|negligent|stubborn|argue|yell)',
        '(rude|disrespectful|impolite|disrespect|(over talk)|(talk over)|unprofessional|negligent|stubborn)(\s|\w+\s|\s\w+\s){0,5}(agent|rep|she|help|he)',
        '(no|zero)(\s|\w+\s|\s\w+\s){0,5}(empathy)'
    ]    
} 


In [32]:
df_agent['cat_agent_dict'] = df_agent["joint_lemma_words"].apply(lambda x:get_category(x))

df_agent["dict_len"] = df_agent["cat_agent_dict"].apply(lambda x: len(x))
df_nohits1= df_agent[df_agent["dict_len"] == 0]
df_agent = df_agent[df_agent["dict_len"] > 0]

df_agent.reset_index(inplace = True, drop=True)

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
#df_temp_cat_agent = 
df_agent_temp = pd.DataFrame(pd.DataFrame(df_agent['cat_agent_dict'].values.tolist()).stack().reset_index(level=1))
#df_temp_bigrams.columns = ['Bigrams', 'bigrams_count']

df_agent_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_agent_temp['hit'] = df_agent_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_agent_temp = pd.concat([df_agent_temp, pd.DataFrame(df_agent_temp['hit'].tolist(), index=df_agent_temp.index)], axis=1)
df_agent_temp['hit'] = df_agent_temp['hit'].apply(lambda x : ' '.join(x))

df_agent = df_agent.join(df_agent_temp)

df_agent.drop(['cat_agent_dict', 'dict_len'], axis = 1, inplace = True)
df_agent['categorization_level'] = 'Agent'
df_agent.head()

,Survey ID,joint_lemma_words,category,hit,0,1,2,3,4,categorization_level
0,13028877gyg-wileska.toledocamacaro@teleperform...,speak staff helpful unfortunately activity pro...,Impolite,be city unprofessional,be,city,unprofessional,None,None,Agent
0,13028877gyg-wileska.toledocamacaro@teleperform...,speak staff helpful unfortunately activity pro...,Unprofessional,be city unprofessional,be,city,unprofessional,None,None,Agent
1,12642167gyg-snider.ochoadelgado@teleperformanc...,absolutely terrific help able rearrange new sc...,Show Expertise,he rearrange new,he,rearrange,new,None,None,Agent
2,13040881gyg-snider.ochoadelgado@teleperformanc...,amazing customer service quick answer solve is...,Knowledgeable,answer solve issue,answer,solve,issue,None,None,Agent
2,13040881gyg-snider.ochoadelgado@teleperformanc...,amazing customer service quick answer solve is...,Issue Resolution,answer solve issue,answer,solve,issue,None,None,Agent


In [33]:
# Brand

categories_dict = {
    "Phone Connectivity": [
        '(lantency|intermittence)(\s|\w+\s|\s\w+\s){0,3}( no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(hear|listen)',
        '( no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(hear|listen)(\s|\w+\s|\s\w+\s){0,3}(lantency|intermittence)'
    ],
    "Order Status Check": [
        '(deliver|courier|driver)(\s|\w+\s|\s\w+\s){0,3}(long|extended|last|prolonged|time)'
    ],
    "Website experience": [
        '(website|link|webpage|page)(\s|\w+\s|\s\w+\s){0,3}(complex|hard|confuse|weird|difficult)', 
        '(website|link|webpage|page)(\s|\w+\s|\s\w+\s){0,3}(crash|error|fail|damage)', 
        '(like|dislike|hate)(\s|\w+\s|\s\w+\s){0,3}(website|link|webpage|page)'   
    ],
    "IVR": [
        '(wait|take|stand|hold|stay)(\s|\w+\s|\s\w+\s){0,3}(long|extended|last|prolonged|time)(\s|\w+\s|\s\w+\s){0,3}(he|she|agent|rep)', 
        '(no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(reach|talk|speak)(\s|\w+\s|\s\w+\s){0,3}(he|she|agent|rep)'
        '(no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(select|find|hear|realize|look|see)(\s|\w+\s|\s\w+\s){0,3}(option|route|chance)',
        '(complex|hard|confuse|weird|difficult)(\s|\w+\s|\s\w+\s){0,3}(understand|hear|use)(\s|\w+\s|\s\w+\s){0,3}(ivr)',
        '(take|wait|stand)(\s|\w+\s|\s\w+\s){0,3}(long|extended|last|prolonged|time)',
        '(long|extended|last|prolonged|time)(\s|\w+\s|\s\w+\s){0,3}(estimate|expect|thing|show)',
    ],
    "Process": [
        '(get|earn|give|issue|pay)(\s|\w+\s|\s\w+\s){0,3}(credit|fee|compensation)',
        '(authenticate|verify|access|confirm|corroborate)(\s|\w+\s|\s\w+\s){0,3}(information|identification|birth|licence|code)',
        '(no|didn|haven|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(receive|see|look|have)(\s|\w+\s|\s\w+\s){0,3}(payment|money|fee|cash)',
        '(no|didn|hasn|wasn| can|(could(\s|\w+\s|\s\w+\s){0,3}no))(\s|\w+\s|\s\w+\s){0,3}(use|turn|utilize)(\s|\w+\s|\s\w+\s){0,3}(cellphone|device|phone)',
        '(cellphone|device|phone)(\s|\w+\s|\s\w+\s){0,3}(5g|network|net|wifi)',
        '(activate|turn|operate|start)(\s|\w+\s|\s\w+\s){0,3}(cellphone|device|phone)'
    ],
    "Escalation Time": [
        '(take|wait|stand|long)(\s|\w+\s|\s\w+\s){0,3}(time|long)(\s|\w+\s|\s\w+\s){0,3}(escalate|transfer)',  
    ],
    "Double Charge": [
        '(charge|pay|discount|payment)(\s|\w+\s|\s\w+\s){0,3}(twice|double)'
    ],
    "Refund": [
        '(refund|compensation|fee|payment)(\s|\w+\s|\s\w+\s){0,3}(wait|long|take|time|last|now)', 
        '(wait|long|take|time|last|now)(\s|\w+\s|\s\w+\s){0,3}(refund|compensation|fee|payment)',
        '(give|get|receive)(\s|\w+\s|\s\w+\s){0,3}(refund|compensation|fee|payment)(\s|\w+\s|\s\w+\s){0,3}(now)'
    ], 
    "Tour Issues": [
        '(tour)(\s|\w+\s|\s\w+\s){0,3}(terrible|aweful|bad|wrong|issues|improve)',
        '(new|register|just)(\s|\w+\s|\s\w+\s){0,3}(never|tour)'
    ],
} 

In [34]:
df_channel['cat_agent_dict'] = df_channel["joint_lemma_words"].apply(lambda x: get_category(x))

df_channel["dict_len"] = df_channel["cat_agent_dict"].apply(lambda x: len(x))
df_nohits2= df_channel[df_channel["dict_len"] == 0]
df_channel = df_channel[df_channel["dict_len"] > 0]
df_channel.reset_index(inplace = True, drop=True)

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
#df_temp_cat_agent = 
df_channel_temp = pd.DataFrame(pd.DataFrame(df_channel['cat_agent_dict'].values.tolist()).stack().reset_index(level=1))
#df_temp_bigrams.columns = ['Bigrams', 'bigrams_count']

df_channel_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_channel_temp['hit'] = df_channel_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_channel_temp = pd.concat([df_channel_temp, pd.DataFrame(df_channel_temp['hit'].tolist(), index=df_channel_temp.index)], axis=1)
df_channel_temp['hit'] = df_channel_temp['hit'].apply(lambda x : ' '.join(x))

df_channel = df_channel.join(df_channel_temp)

df_channel.drop(['cat_agent_dict', 'dict_len'], axis = 1, inplace = True)
df_channel['categorization_level'] = 'Brand'
df_channel.head()

,Survey ID,joint_lemma_words,category,hit,0,1,2,3,4,categorization_level
0,13002836gyg-karla.chungvelasquez@teleperforman...,understand charge twice booking accidentally a...,Double Charge,charge twice,charge,twice,None,None,None,Brand
0,13002836gyg-karla.chungvelasquez@teleperforman...,understand charge twice booking accidentally a...,Refund,take account fee,take,account,fee,None,None,Brand
1,13046780gyg-wileska.toledocamacaro@teleperform...,request refund ask cancel booking operate need...,Refund,wait refund,wait,refund,None,None,None,Brand
2,12674499gyg-carmen.canalesore@teleperformance....,wait return discuss request refund ghost tour ...,Refund,wait request refund,wait,request,refund,None,None,Brand
3,13065430gyg-grecia.romanneyra@teleperformance....,wait period pretty long matter need meet resolve,IVR,wait pretty long,wait,pretty,long,None,None,Brand


In [35]:
df = pd.concat([df_agent, df_channel])
df.drop(['joint_lemma_words',0,1,2,3,4], axis = 1, inplace = True)
df.categorization_level.value_counts(dropna=False)



Agent    1065
Brand     189
Name: categorization_level, dtype: int64

In [36]:
df_sentiment=pd.read_csv('output/Sentiment.csv', sep=';')

In [37]:
df = df.merge(df_sentiment, on='Survey ID')
df.to_csv('output/Categorization.csv', index=False, sep=';')

In [38]:
# Detractors Categorization

In [39]:
df2_csat = data.loc[:, ['Survey ID', 'CSAT']]
merged_df = pd.merge(df_cat, df2_csat, on='Survey ID', how='left')
df_detractor=merged_df[(merged_df['CSAT']=='Passive') | (merged_df['CSAT']=='Detractor')]
df_promoter=merged_df[merged_df['CSAT']=='Promoter']



In [40]:
categories_dict = {
    "Clarity" : [
        '(clear|unclear|cl.ar|cle.r|uncl.ar|uncle.r)(\s+\w+){0,3}(service|ask|intelligent|answer|response|information)',
        '((clear|unclear|cl.ar|cle.r|uncl.ar|uncle.r)\s+)?(service(\s+\w+)?|\w+(\s+\w+)?\s+unclear)',
        'hard|complicated|impossible(\s+\w+){0,3}\s+understand',
        'clearly(\s+\w+){0,3}chat|person?(\s+\w+){0,3}chat|person?',
        '(lack|bad|wrong|could)?(\s+\w+){0,3}(understanding|understand)(\s+\w+){0,3}(english)?(\s+\w+){0,3}(language)?',
        '(no\s+entiendo|difícil\s+de\s+entender|poco\s+claro)(\s+\w+){0,3}(información|respuesta|servicio|ayuda)',
        '(confuso|confundido|incomprensible|oscuro)(\s+\w+){0,3}(información|respuesta|servicio|ayuda)',
        'ayuda(\s+\w+){0,3}(confusa|confundida|poco\s+clara)',
        '(respuesta|información|ayuda|servicio)(\s+\w+){0,3}(ambigua|ambiguo|vaga|vago)',
        '(respuesta|información|ayuda|servicio)(\s+\w+){0,3}(incompleta|incompleto)',
        '(respuesta|información|ayuda|servicio)(\s+\w+){0,3}(incorrecta|incorrecto)',
        'ambiguo|vago(\s+\w+){0,3}(respuesta|información|ayuda|servicio)',
        'incompleto(\s+\w+){0,3}(respuesta|información|ayuda|servicio)',
        'incorrecto(\s+\w+){0,3}(respuesta|información|ayuda|servicio)'
],
    "Problem Solving":[
        '(poor|waste|bad)(\s\w+){0,3}(service)', 
        '(time)(\s\w+){0,3}(waste)',  
        '(resolve|fix|help|assist)(\s\w+){0,3}(problem)', 
        '(unable)(\s\w+){0,3}(help)', 'cause(\s\w+){0,3}trouble',  
        '(issue|problem)(\s\w+){0,3}(not(\s\w+){0,3}solved|unsolved|unresolved|unfixed|unresolved)',   
        '(solving|unsolving|unresolving|not(\s\w+){0,3}fixing|unfixing)(\s\w+){0,3}(problem|issue)',
        '(problem|issue)(\s\w+){0,3}(ignored|neglected|unattended|unaddressed)',  
        '(useless|worthless|ineffective|unhelpful|unproductive)(\s\w+){0,3}(help|assistance)',  
        '(did(\s\w+){0,3}nothing|nothing(\s\w+){0,3}(done|accomplished)|no(\s\w+){0,3}action(\s\w+){0,3}taken)',
        '(support|service)(\s\w+){0,3}(not(\s\w+){0,3}working|broken|useless|ineffective)', 
        '(able)?(\s\w+){0,3}(solve|fix|help|assist)(\s\w+){0,3}(issue|problem)',
        '(help|assist|solve|fix|resolve)(\s\w+){0,3}(problem|issue)',
        '(useless|worthless|ineffective|unhelpful|unproductive)(\s\w+){0,3}(support|service)',
         '(servicio\s)?(pobre|malo|deficiente|insuficiente|ineficaz)',
        'tiempo(\s\w+){0,3}(perdido|desperdiciado)',
        '(solucionar|arreglar|ayudar|asistir)(\s\w+){0,3}(problema)',
        '(no\s)?(capaz|poder)(\s\w+){0,3}(ayudar|solucionar)',
        'caus(\s\w+){0,3}(problemas|inconvenientes)',
        'respuestas(\s\w+){0,3}(lentas|tardadas)',
        'atención(\s\w+){0,3}(deficiente|pobre|insuficiente|ineficaz)',
        '(mala|pobre|deficiente|insuficiente|ineficaz)(\s\w+){0,3}(comunicación|respuesta)',
        '(no\s)?(resolver|solucionar|arreglar)(\s\w+){0,3}(solicitud|problema)',
        '(no\s)?(ayudar|asistir|brindar)(\s\w+){0,3}(solución|respuesta)'
        
    ],
    "Knowledge":[
        '(incorrect|wrong|bad|false)?(\s|\w+\s|\s\w+\s){0,3}(information|problem)',
        '(completely)(\s|\w+\s|\s\w+\s){0,3}(false|accurate)',
        '\b(lie)\b',
        '(incorrecta|equivocada|falsa)?(\s|\w+\s|\s\w+\s){0,3}(información|dato|problema)',
        '(completamente)(\s|\w+\s|\s\w+\s){0,3}(falso|preciso)',
        '\b(mentira)\b',
        '(saber)(\s|\w+\s|\s\w+\s){0,3}(respuesta)',
        '(información incorrecta|dato incorrecto|información errónea)',
        '(información equivocada|dato equivocado|información incorrecta)',
        '(datos incorrectos|datos erróneos)',
        '(mala información|mala respuesta)',
        r'(incorrect|wrong|bad|false)?(\s|\w+\s|\s\w+\s){0,3}(information|problem)',
        r'(completely)(\s|\w+\s|\s\w+\s){0,3}(false|accurate)',
        r'(lie)',
        r'(know)(\s|\w+\s|\s\w+\s){0,3}(nothing|answer)',
        r'(incorrect|wrong)(\s|\w+\s|\s\w+\s){0,3}(information|info)',
    ],
    "Responsiveness" : [
        '(long|forever|too(\s\w+){0,3}long|lengthy)(\s\w+){0,3}(wait|hold|response)',
        '(disconnected|disconnection|drop|dropped)(\s\w+){0,3}(chat|call)',
        'hang(\s\w+){0,3}(up|on)?',
        'took(\s\w+){0,3}(forever|too(\s\w+){0,3}long)',
        '(waiting(\s\w+){0,3}time|time(\s\w+){0,3}(wait|hold|response))(\s\w+){0,3}(long|forever|too(\s\w+){0,3}long|lengthy)',
        '(service(\s\w+){0,3}(delayed|taking(\s\w+){0,3}long)|response(\s\w+){0,3}(time|delay))(\s\w+){0,3}(long|forever|too(\s\w+){0,3}long|lengthy)',
        "long(\s\w+){0,3}wait",
        "too(\s\w+){0,3}long(\s\w+){0,3}(chat|response|time)",
        "waited(\s\w+){0,3}(hours|minutes|time)(\s\w+){0,3}(hold|on hold|for a response)",
        "disconnected(\s\w+){0,3}chat",
        "hung(\s\w+){0,3}up(\s\w+){0,3}(chat|conversation)",
        "ignored(\s\w+){0,3}(chat|conversation|message)",
        "(chat|response)(\s\w+){0,3}(delayed|slow|taking(\s\w+){0,3}forever|not(\s\w+){0,3}responding)",
        "(response|chat)(\s\w+){0,3}(time|speed)(\s\w+){0,3}(too(\s\w+){0,3}slow|unacceptable|poor|bad)",
        '(espera(\s+\w+){0,3}(larga|demasiado(\s+\w+){0,3}larga|interminable)|tiempo(\s+\w+){0,3}(de\s+espera|de\s+respuesta)(\s+\w+){0,3}(largo|demasiado(\s+\w+){0,3}largo|interminable))',
        '(desconectado|desconexión|caída|caído)(\s+\w+){0,3}(chat|llamada|conexión)',
        'colg(ó|ando)(\s+\w+){0,3}(el\s+teléfono|en\s+el\s+chat)?',
        'tard(ó|ando)(\s+\w+){0,3}(demasiado|una\s+eternidad)',
        '(demora|retraso)(\s+\w+){0,3}(en\s+el\s+servicio|en\s+la\s+respuesta|en\s+responder|en\s+contestar)',
        '(tiempo(\s+\w+){0,3}espera)'
    ],

    
    'Friendliness': [
       '(friendly(\s+\w+){0,1}\s+lack|lack(\s+\w+){0,1}\s+friendly)',
        '\b(rude)\b.*?\b(customer service|agent|representative|support)\b',
        "friendly(\\s+\\w+){0,1}\\s+lack|lack(\\s+\\w+){0,1}\\s+friendly",
        "\\b(rude)\\b.*?\\b(agent|person|rep|representative|support)\\b",
        "(un)?(empathetic|empathy|empathize)(\\s+\\w+){0,3}(agent|person|rep|representative|support)?",
        "(condescending|patronizing)(\\s+\\w+){0,3}(agent|person|rep|representative|support)?",
        "took(\\s+\\w+){0,3}(seriously|serious)(\\s+\\w+){0,3}(agent|person|rep|representative|support)?",
        "over(?:-|\\s)?talk(\\s+\\w+){0,3}(agent|person|rep|representative|support)?",
        "apologize(\\s+\\w+){0,3}(agent|person|rep|representative|support)?",
        "(polite|courteous|respectful)(\\s+\\w+){0,3}(agent|person|rep|representative|support)?",
        '(falta(\s+\w+){0,1}\s+de(\s+\w+){0,1}\s+amabilidad|amabilidad(\s+\w+){0,1}\s+falta)',
        '\b(mal|groser|descortés)\b.?\b(servicio al cliente|agente|representante|soporte)\b',
        "amabilidad(\s+\w+){0,1}\s+falta|falta(\s+\w+){0,1}\s+amabilidad",
        "\b(mal|groser|descortés)\b.?\b(agente|persona|representante|soporte)\b",
        "(falta de )?(empatía|empático|empatizar)(\s+\w+){0,3}(agente|persona|representante|soporte)?",
        "(condescendiente|paternalista)(\s+\w+){0,3}(agente|persona|representante|soporte)?",
        "no tom(ó|aron)(\s+\w+){0,3}(en serio|en cuenta)(\s+\w+){0,3}(agente|persona|representante|soporte)?",
        "(interrumpir|hablar de más|no dejar hablar)(\s+\w+){0,3}(agente|persona|representante|soporte)?",
        "disculparse(\s+\w+){0,3}(agente|persona|representante|soporte)?",
        "(político|cortés|respetuoso)(\s+\w+){0,3}(agente|persona|representante|soporte)?"
    ],
    'Professionalism':
    [
        r'\b(unprofessional|unethical|unacceptable|inappropriate)\b.*?\b(service|behavior|conduct|attitude)\b',
        r'\b(service|behavior|conduct|attitude)\b.*?\b(unprofessional|unethical|unacceptable|inappropriate)\b',
        r'\b(agent|representative|support)\b.*?\b(professional|polite|courteous|respectful|helpful)\b',
        r'\b(professional|polite|courteous|respectful|helpful)\b.*?\b(agent|representative|support)\b',
        r'(?i)\b(emoji|emoticon)\b.*?\b(unprofessional|inappropriate|misinterpreted|misused)\b',
        r'(?i)\b(unprofessional|inappropriate|misinterpreted|misused)\b.*?\b(emoji|emoticon)\b',
        r'\b(agente|representante|soporte)\b.*?\b(profesional|educado|cortés|respetuoso|servicial)\b',
        r'\b(profesional|educado|cortés|respetuoso|servicial)\b.*?\b(agente|representante|soporte)\b',
        r'(?i)\b(emoji|emoticono)\b.*?\b(poco profesional|inapropiado|malinterpretado|mal utilizado)\b',
        r'(?i)\b(poco profesional|inapropiado|malinterpretado|mal utilizado)\b.*?\b(emoji|emoticono)\b'
]


}

In [41]:
df_detractor['cat_dict'] = df_detractor['joint_lemma_words'].apply(lambda x:get_category(str(x)))


df_detractor['dict_len'] = df_detractor['cat_dict'].apply(lambda x: len(x))
df_detractor=df_detractor[df_detractor['dict_len']>0]
df_detractor.drop('dict_len', axis=1, inplace=True)
df_detractor.reset_index(inplace=True, drop=True)



df_detractor_temp = pd.DataFrame(pd.DataFrame(df_detractor['cat_dict'].values.tolist()).stack().reset_index(level=1))
df_detractor_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_detractor_temp['hit'] = df_detractor_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_detractor_temp = pd.concat([df_detractor_temp, pd.DataFrame(df_detractor_temp['hit'].tolist(), index=df_detractor_temp.index)], axis=1)
df_detractor_temp['hit'] = df_detractor_temp['hit'].apply(lambda x : ' '.join(x))

df_detractor = df_detractor.join(df_detractor_temp)

df_detractor.drop(['cat_dict'], axis = 1, inplace = True)
df_detractor['categorization_level'] = 'Dissatisfier'
df_detractor.drop('joint_lemma_words', inplace=True, axis=1)
df_detractor.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Survey ID,CSAT,category,hit,0,1,2,categorization_level
0,13002836gyg-karla.chungvelasquez@teleperforman...,Detractor,Clarity,understand booking cancel,understand,booking,cancel,Dissatisfier
1,12977594gyg-grecia.romanneyra@teleperformance....,Detractor,Clarity,request,request,None,None,Dissatisfier
2,13069495gyg-pablo.amasifuensacsa@teleperforman...,Detractor,Clarity,understand policy want,understand,policy,want,Dissatisfier
3,13040466gyg-andre.lopezsenitagoya@teleperforma...,Detractor,Clarity,,None,None,None,Dissatisfier
4,13084100gyg-alvaro.floresuribe@teleperformance...,Passive,Clarity,sitka,sitka,None,None,Dissatisfier


In [42]:
df_detractor = df_detractor.merge(df_sentiment, on='Survey ID')
df_detractor.to_csv('output/DSAT_Categorization.csv', index=False, sep=';')

In [44]:
from datetime import datetime as dt 

print('Updated on', dt.now())


Updated on 2023-06-01 09:59:01.259347
